In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np

import spacy
#!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

import nltk
nltk.download('stopwords')

     |████████████████████████████████| 777.4 MB 16 kB/s  eta 0:00:013   |█▎                              | 31.6 MB 451 kB/s eta 0:27:34     |██████▌                         | 156.9 MB 1.0 MB/s eta 0:09:58     |██████████████████████▊         | 551.2 MB 274 kB/s eta 0:13:44     |███████████████████████▉        | 579.0 MB 459 kB/s eta 0:07:12     |████████████████████████████▎   | 686.3 MB 219 kB/s eta 0:06:55     |███████████████████████████████▊| 771.0 MB 465 kB/s eta 0:00:14
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dragonzord/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
patient_notes = pd.read_csv("patient_notes.csv")
print(patient_notes.shape)
patient_notes.head(2)

(42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."


In [112]:
patient_notes[patient_notes["case_num"] == 0].head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."
2,2,0,"Dillon Cleveland is a 17 y.o. male patient with no significant PMH who presents with complaints of heart pounding. This has been going on for a few months and happens once or twice a month. He cannot think of any triggers, and it has occurred both with activity and at rest. Occasionally, it is accompanied by chest pressure but not pain that is located at the center of his chest. On one instance, he experienced chest pressure, lightheadedness and shortness of breath for 10 minutes with the heart pounding. \r\n\r\nOtherwise, he has not had shortness of breath, chest pain, anxiety, \r\n\r\nMedication: adderall twice a week as study aid (not prescribed)\r\nFH: mother - thyroid disease, father - heart attack at age 52, both living\r\nSH: no smoking; 3-4 drinks on the weekend per sitting; marijuanna once recently, no other recreational drugs"
3,3,0,"a 17 yo m c/o palpitation started 3 mos ago; \r\nNOTHING IMPROVES OR EXACERBATES THE SYMPTOMS ACCORDING TO HIM; IT CAN HAPPEN ANY TIME; MAY TAKE A FEW MINUTES; LAST TIME HAPPENED 2 DAYS AGO DURING PLAYING A GAME AND IT WAS ASSOCIATED WITH RETROSTERNAL PRESSURE LIKE DISCOMFORT; AND HE FELT LIGHTHEADED. BUT HE DID NOT LOSE CONCIOUSNESS AND DID NOT FALL. \r\nNOT ASSOCIATED WITH NAUSEA VOMITING; HEADACHE; ABDOMINAL PAIN; CHANGES IN URINATION OR BOWEL HABITS, OR TREMOR OR SKIN OR HAIR CHANGE OR INTOLERABC\r\n\r\nPMH NONE\r\nPSHH: NONE\r\nMEDS: ADEROL TO STAY AWAKE\r\nHOSP: NONE\r\nFH: MOTHER HAS THYROID DISEASE; FATHER HAS CAD X 1 YR\r\nSH: NO SMOKING; DRINKING ON WEEKENDS CAGE 0/4; ONE TIME USE OF MARIJUANA\r\n\r\n\r\n"
4,4,0,"17yo male with no pmh here for evaluation of palpitations. States for the last 3-4mo he has felt that his heart with intermittently ""beat out of his chest,"" with some associated difficulty catching his breath. States that the most recent event was 2 days ago, and during activity at a soccer game. He does not seem to note any specific precipitatinig factors at this time. He also states that he feels as if he will faint during these events, but has not lost consciousness at any point. Furthermore, he does endorse theses attacks occuring 1-2 times a month and peak at 4 mins. He denies any stressors at home. ROS: denies weight loss, fevers, recnet illness, change in bowel habits. PMH: negative, PSH negative, FHX mom with thyroid disorder, dad with heart condition and MI at 52yo. SHX no tobacco, ETOH on weekends, Marijuana tried a month ago. Med: is taking some of roommates Adderoll intermittently (last was 2 days ago prior to event). KNDA"


In [40]:
features = pd.read_csv("features.csv")
print(features.shape)
features.head(2)

(143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction
1,1,0,Family-history-of-thyroid-disorder


In [117]:
features.count()

feature_num     143
case_num        143
feature_text    143
dtype: int64

In [165]:
features.feature_num.value_counts()

0      1
611    1
605    1
606    1
607    1
      ..
305    1
306    1
307    1
308    1
916    1
Name: feature_num, Length: 143, dtype: int64

In [119]:
features.case_num.value_counts()

5    18
8    18
2    17
9    17
3    16
0    13
1    13
6    12
4    10
7     9
Name: case_num, dtype: int64

In [116]:
features[(features["case_num"] == 0) & (features["feature_num"] == 1)]

,feature_num,case_num,feature_text
1,1,0,Family-history-of-thyroid-disorder


### Dictionary of case numbers and the phrases for which to look

In [173]:
case_num_feature_text = {}
for i in range(10):
    case_num_feature_text[i] = (features["feature_text"][features["case_num"] == i]).to_list()
case_num_feature_text

{0: ['Family-history-of-MI-OR-Family-history-of-myocardial-infarction',
  'Family-history-of-thyroid-disorder',
  'Chest-pressure',
  'Intermittent-symptoms',
  'Lightheaded',
  'No-hair-changes-OR-no-nail-changes-OR-no-temperature-intolerance',
  'Adderall-use',
  'Shortness-of-breath',
  'Caffeine-use',
  'heart-pounding-OR-heart-racing',
  'Few-months-duration',
  '17-year',
  'Male'],
 1: ['No-vaginal-discharge',
  'Weight-loss',
  'Not-sexually-active',
  'Prior-episodes-of-diarrhea',
  '20-year',
  'No-bloody-bowel-movements',
  'Recurrent-bouts-over-past-6-months',
  'Right-sided-LQ-abdominal-pain-OR-Right-lower-quadrant-abdominal-pain',
  'No-urinary-symptoms',
  'Diminished-appetite',
  'Normal-LMP-2-weeks-ago-OR-Normal-last-menstrual-period-2-weeks-ago',
  '8-to-10-hours-of-acute-pain',
  'Female'],
 2: ['Prior-normal-periods',
  'Last-Pap-smear-I-year-ago',
  'IUD',
  'Sexually-active',
  'Vaginal-dryness',
  'Irregular-menses',
  'Recent-nausea-vomiting-OR-Recent-flulike-sy

In [ ]:
features.feature_text.value_counts().to_dict()

In [ ]:
notes_features[["feature_num", "feature_text", "case_num"]][(notes_features["feature_num"] == 1)]

In [ ]:
notes_features = patient_notes.merge(right=features, left_on="case_num", right_on="case_num")#.drop("case_num_y", axis=1).rename({"case_num_x": "case_num"}, axis=1)
print(notes_features.shape)
notes_features.head()

In [ ]:
notes_features.isnull().sum()

In [ ]:
notes_features[notes_features["case_num"] == 0].head()

In [138]:
notes_features["feature_num"].value_counts()

305    9753
308    9753
300    9753
301    9753
302    9753
       ... 
107     808
108     808
112     808
111     808
100     808
Name: feature_num, Length: 143, dtype: int64

In [139]:
notes_features["case_num"].value_counts()

3    156048
5    124362
9     87567
8     75528
4     54050
7     36909
2     33286
0     29484
6     19164
1     10504
Name: case_num, dtype: int64

In [150]:
len(notes_features["case_num"].value_counts().index)

10

In [ ]:
feature_dict = {i: [(notes_features["feature_text"][notes_features["case_num"] == i]).values] for i in range(len(notes_features["case_num"].value_counts().index))}
feature_dict

In [ ]:
notes_features["pn_history"][notes_features["case_num"] == 0][:10]

In [141]:
case_num_zero = notes_features["pn_history"][notes_features["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
whole_case_num_zero[:1958]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std\n17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\

### brents acquire

In [142]:
import acquire

### jareds prepare file

In [143]:
import prepare_jag
clean_note = prepare_jag.basic_clean3(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

966764

### Using spacy

In [2]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(clean_note)

NameError: name 'spacy' is not defined

In [82]:
print(len(doc))
doc[:200]

158876


17-year-old male come student health clinic complaining heart pounding mr cleveland's mother given verbal consent history physical examination treatment -began 2-3 months ago sudden intermittent 2 days lasting 3-4 min worsening non-allev/aggrav -associated dispnea exersion rest stressed school -reports fe feels like heart jumping chest -ros denies chest pain dyaphoresis wt loss chills fever nausea vomiting pedal edeam -pmh non meds aderol friend nkda -fh father mi recently mother thyroid dz -sh non-smoker mariguana 5-6 months ago 3 beers weekend basketball school -sh no std 17 yo male recurrent palpitations past 3 mo lasting 3 - 4 min happened 5 - 6 times since beginning one time durign baskeball game two days ago light headedness pressure chest catching breath no fainting teh episodes no sweating no diarrhea no heat intolerance no weight loss tried aterol able better concentrate received roommate dillon cleveland 17 male patient no significant pmh presents complaints heart pounding go

### This might be the way to go to find locations

In [83]:
for token in doc[:200]:
    if token.pos_ not in ["NUM", "PUNCT", "SYM"] and token.ent_type_ not in ["DATE", "TIME"]:
        print(token.text, token.ent_iob_, token.ent_type_, token.pos_)

male O  NOUN
come O  VERB
student O  NOUN
health O  NOUN
clinic O  NOUN
complaining O  VERB
heart O  NOUN
pounding O  VERB
mr O  PROPN
cleveland B GPE PROPN
's O  PART
mother O  NOUN
given O  VERB
verbal O  ADJ
consent O  NOUN
history O  NOUN
physical O  ADJ
examination O  NOUN
treatment O  NOUN
sudden O  ADJ
intermittent O  ADJ
lasting O  VERB
worsening O  VERB
non O  ADJ
- O  ADJ
allev O  ADJ
aggrav O  PROPN
-associated O  VERB
dispnea O  PROPN
exersion O  NOUN
rest O  NOUN
stressed O  VERB
school O  NOUN
-reports O  NOUN
fe O  PROPN
feels O  VERB
like O  ADP
heart O  NOUN
jumping O  VERB
chest O  NOUN
-ros O  PROPN
denies O  VERB
chest O  NOUN
pain O  NOUN
dyaphoresis O  NOUN
wt O  NOUN
loss O  NOUN
chills O  NOUN
fever O  NOUN
nausea O  NOUN
vomiting O  NOUN
pedal O  NOUN
edeam O  NOUN
-pmh O  ADJ
non O  ADJ
meds O  NOUN
aderol O  VERB
friend O  PROPN
nkda O  NOUN
father O  PROPN
mi B PERSON PROPN
recently O  ADV
mother O  NOUN
thyroid O  PROPN
dz O  PROPN
non O  ADJ
- O  ADJ
smoke

In [1]:
for token in doc[:100].noun_chunks:
    print(token)
    print({tok.text: tok.pos_ for tok in token if tok.pos_ not in ["NUM", "PUNCT", "SYM", "DATE"]})

NameError: name 'doc' is not defined

In [ ]:
# colors = {"Annotation" :"linear-gradient(to right, #2980b9, #6dd5fa, #ffffff);" } 
# options = {"colors": colors}
#spacy.displacy.render(doc, style='ent', options=options , manual=True, jupyter=True);
spacy.displacy.render(doc, style='ent', jupyter=True);

In [ ]:
# for ent in doc[:10000].ents:
#     print(f"text: {ent.text}, start_char: {ent.start_char}, end_char: {ent.end_char}, label_: {ent.label_}, start: {ent.start}, ent_id: {ent.ent_id_}, label: {ent.label}, vector_norm: {ent.vector_norm}")

In [ ]:
# for text, labels in data:
#         doc=[]
#         #print(text)
#         doc = nlp(text)
    
#         ents = []
#         labels["entities"]=sorted(labels["entities"])
#         for start, end, label in labels["entities"]:
#             print(start,end,label)
#             ents.append(doc.char_span(start, end, label))
        
#         print(ents)